<a href="https://colab.research.google.com/github/AffanSurya/SenakApp_ML/blob/main/Model/Model_Recomandations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install tensorflow_recommenders

In [14]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from typing import Dict, Text

import warnings
warnings.filterwarnings('ignore')

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
df_recipes = pd.read_csv('/content/drive/MyDrive/Dataset/recipes_reduce.csv')
df_reviews = pd.read_csv('/content/drive/MyDrive/Dataset/reviews_reduce.csv')

In [17]:
df_recipes.info()
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23899 entries, 0 to 23898
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    23899 non-null  int64  
 1   Name                        23899 non-null  object 
 2   AuthorId                    23899 non-null  int64  
 3   Description                 23899 non-null  object 
 4   Images                      23899 non-null  object 
 5   RecipeCategory              23891 non-null  object 
 6   RecipeIngredientQuantities  23899 non-null  object 
 7   RecipeIngredientParts       23899 non-null  object 
 8   AggregatedRating            23898 non-null  float64
 9   ReviewCount                 23899 non-null  float64
 10  Calories                    23899 non-null  float64
 11  Fat                         23899 non-null  float64
 12  SaturatedFat                23899 non-null  float64
 13  Cholesterol                 238

In [18]:
df_recipes.head()

,RecipeId,Name,AuthorId,Description,Images,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,Fat,SaturatedFat,Cholesterol,Sodium,Carbohydrate,Fiber,Sugar,Protein,RecipeInstructions
0,40,Best Lemonade,1566,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,"c(""Into a 1 quart Jar with tight fitting lid, ..."
1,42,Cabbage Soup,1538,Make and share this Cabbage Soup recipe .,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,"c(""Mix everything together and bring to a boil..."
2,44,Warm Chicken A La King,1596,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",Chicken,"c(""12"", ""2"", ""3"", ""450"", ""1"", ""2"", ""1/4"", ""1"",...","c(""chicken"", ""butter"", ""flour"", ""milk"", ""celer...",5.0,23.0,895.5,66.8,31.9,405.8,557.2,29.1,3.1,5.0,45.3,"c(""Melt 1 1/2 ozs butter, add the flour and co..."
3,49,Chicken Breasts Lombardi,174711,Make and share this Chicken Breasts Lombardi r...,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""2"", ""2"", ""12"", ""1/2"", ""1/3"", ""3/4"", ""1/2"", ...","c(""fresh mushrooms"", ""butter"", ""boneless skinl...",5.0,21.0,627.7,24.9,13.0,203.0,848.9,13.7,0.6,2.0,57.9,"c(""Cook mushrooms in 2 tbsp butter in a large ..."
4,54,Carrot Cake,1535,This is one of the few recipes my husband ever...,"c(""https://img.sndimg.com/food/image/upload/w_...",Dessert,"c(""3"", ""4"", ""1 1/4"", ""2"", ""2"", ""2"", ""2"", ""1"", ...","c(""carrots"", ""eggs"", ""white sugar"", ""all-purpo...",5.0,17.0,522.6,27.1,4.9,69.8,534.8,67.0,1.6,47.9,5.0,"c(""Beat together the eggs, oil, and white suga..."


In [19]:
df_reviews.head()

,ReviewId,Name,RecipeId,AuthorId,Rating,Review,DateSubmitted,DateModified
0,2,Jalapeno Pepper Poppers,992,2008,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,13,Kevin's Best Corned Beef,7435,1773,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
2,14,Warm Chicken A La King,44,2085,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
3,17,Spinach and Strawberry Salad,5221,2046,4,"love it, but without the bean sprouts.",2000-05-08T11:08:00Z,2000-05-08T11:08:00Z
4,19,Neiman-Marcus $250 Chocolate Chip Cookies Recipe,13307,2046,5,"chewy goodness, not crispy at all. i even thre...",2000-05-21T16:59:00Z,2000-05-21T16:59:00Z


In [20]:
df_reviews = df_reviews.astype({'AuthorId': 'string'})


Reviews = tf.data.Dataset.from_tensor_slices(dict(df_reviews[['AuthorId', 'Name', 'Rating']]))
Recipes = tf.data.Dataset.from_tensor_slices(dict(df_recipes[['Name']]))

Reviews = Reviews.map(lambda x: {
    "Name": x["Name"],
    "AuthorId": x["AuthorId"],
    "Rating": float(x["Rating"])
})

Recipes = Recipes.map(lambda x: x["Name"])

In [21]:
print('Total Data: {}'.format(len(Reviews)))

tf.random.set_seed(42)
shuffled = Reviews.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = Reviews.take(35_000)
test = Reviews.skip(35_000).take(8_188)

Total Data: 724541


In [22]:
food_titles = Recipes.batch(1_000)
user_ids = Reviews.batch(1_000).map(lambda x: x["AuthorId"])

unique_food_titles = np.unique(np.concatenate(list(food_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Recipes: {}'.format(len(unique_food_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Recipes: 23899
Unique users: 185252


In [50]:
class FoodModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()

        embedding_dimension = 64

        self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])

        self.food_model: tf.keras.layers.Layer = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_food_titles, mask_token=None),
            tf.keras.layers.Embedding(len(unique_food_titles) + 1, embedding_dimension)
        ])

        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(1)
        ])

        # Task ranking dan retrieval ditambahkan sebagai atribut
        self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

        self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval()

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        user_embeddings = self.user_model(features["AuthorId"])
        food_embeddings = self.food_model(features["Name"])

        return (
            user_embeddings,
            food_embeddings,
            self.rating_model(
                tf.concat([user_embeddings, food_embeddings], axis=1)
            ),
        )

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        ratings = features.pop("Rating")

        user_embeddings, food_embeddings, rating_predictions = self(features)

        # Menggunakan self.rating_task yang sudah ditambahkan sebelumnya
        rating_loss = self.rating_task(
            labels=ratings,
            predictions=rating_predictions,
        )

        retrieval_loss = self.retrieval_task(user_embeddings, food_embeddings)

        # And combine them using the loss weights.
        return rating_loss + retrieval_loss


In [51]:
# Create an instance of the model
model = FoodModel()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

# Prepare the training and test datasets
cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

# Train the model
model.fit(cached_train, epochs=3)

Epoch 1/3
35/35 [==============================] - 3s 31ms/step - root_mean_squared_error: 1.9974 - loss: 6910.9912 - regularization_loss: 0.0000e+00 - total_loss: 6910.9912
Epoch 2/3
35/35 [==============================] - 1s 27ms/step - root_mean_squared_error: 0.9967 - loss: 6732.4501 - regularization_loss: 0.0000e+00 - total_loss: 6732.4501
Epoch 3/3
35/35 [==============================] - 2s 52ms/step - root_mean_squared_error: 1.1056 - loss: 6071.7049 - regularization_loss: 0.0000e+00 - total_loss: 6071.7049


In [62]:
metrics = model.evaluate(cached_test, return_dict=True)

# Rating task metrics
rating_rmse = metrics['root_mean_squared_error']

# Retrieval task metrics
retrieval_accuracy = metrics.get('accuracy', 'N/A')

print(f"\nRating RMSE: {rating_rmse:.3f}")
print(f"Retrieval accuracy: {retrieval_accuracy}")


9/9 [==============================] - 0s 30ms/step - root_mean_squared_error: 1.0852 - loss: 5832.1992 - regularization_loss: 0.0000e+00 - total_loss: 5832.1992

Rating RMSE: 1.085
Retrieval accuracy: N/A


In [66]:
# Function to get the first image URL from the 'Images' column
def get_image_url(recipe_df, recipe_name, image_column='Images'):
    image_urls = recipe_df[recipe_df['Name'] == recipe_name][image_column].values
    if len(image_urls) > 0:
        # Using regular expression to extract the first URL
        match = re.search(r'https://.*?\.(jpg|JPG|png)', image_urls[0])
        if match:
            return match.group()
    return None

# Select a random user from unique_user_ids
random_user_id = df_reviews['AuthorId'].sample(1).values[0]

def recommend_food_for_random_user(model, top_n=20):
    k = top_n  # Set k to the desired number of recommendations
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    index.index_from_dataset(
        tf.data.Dataset.zip((Recipes.batch(100), Recipes.batch(100).map(model.food_model)))
    )

    _, titles = index(tf.constant([str(random_user_id)]), k=k)
    recommended_titles = [title.decode("utf-8") for title in titles[0].numpy()]
    return recommended_titles


# Call the function to recommend food for a random user
recommended_titles = recommend_food_for_random_user(model, top_n=40)

# Print the recommended products for the random user
print("Top recommended products for User {}: ".format(random_user_id))
for i, title in enumerate(recommended_titles):
    image_url = get_image_url(df_recipes, title)
    print('{}. {}\n   Image URL: {}\n'.format(i+1, title, image_url))


Top recommended products for User 2001836029: 
1. Crock Pot Stuffed Chicken Rolls
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/21/77/1/xbdImPNQCCUNX2CneM4Q_chicken-roll-ups-4.jpg

2. Cracker Barrel's Hashbrowns Casserole - Copycat
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/15/24/2/WHQJZyXcSvqGFnAl5QEf_cracker-barrell-hash-brown-casserole_1794.jpg

3. Crock Pot Chops That Will Make You Eat Til You Pop!
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/29/54/4/pic3IiYqW.jpg

4. Wonderful Salsa
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/92/72/57kV2MbdSregy8TZfkHj_batch23-6580.jpg

5. Crock Pot Peach Dump Dessert
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/28/7/SliJNBLXRkKWbfYP

In [67]:
random_user_id = df_reviews['AuthorId'].sample(1).values[0]

def predict_food_for_random_user(top_n=2):
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    index.index_from_dataset(
      tf.data.Dataset.zip((Recipes.batch(100), Recipes.batch(100).map(model.food_model)))
    )

    _, titles = index(tf.constant([str(random_user_id)]))

    print('Top {} recommendations for user {}:\n'.format(top_n, random_user_id))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

predict_food_for_random_user(top_n=2)


Top 2 recommendations for user 237624:

1. Betty White's Chicken Wings
2. Pecan Pie Muffins


In [68]:
random_user_id = df_reviews['AuthorId'].sample(1).values[0]

def recommend_food_for_random_user(model, top_n):
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    index.index_from_dataset(
        tf.data.Dataset.zip((Recipes.batch(100), Recipes.batch(100).map(model.food_model)))
    )

    _, titles = index(tf.constant([str(random_user_id)]))

    recommended_titles = [title.decode("utf-8") for title in titles[0, :top_n].numpy()]
    return recommended_titles

# Function to get image URL for a given recipe name
def get_image_url(recipe_df, recipe_name, image_column='Images'):
    image_urls = recipe_df[recipe_df['Name'] == recipe_name][image_column].values
    if len(image_urls) > 0:
        match = re.search(r'https://.*?\.(jpg|JPG)', image_urls[0])
        if match:
            return match.group()
    return None

# Function to recommend food and display images
def recommend_food_with_images(model, recipe_df, top_n):
    recommended_titles = recommend_food_for_random_user(model, top_n=top_n)

    print('Top {} recommendations for user {}:\n'.format(top_n, random_user_id))
    for i, title in enumerate(recommended_titles):
        image_url = get_image_url(recipe_df, title)
        print('{}. {}\n   Image URL: {}\n'.format(i+1, title, image_url))

# Call the function to recommend food with images
recommend_food_with_images(model, df_recipes, 20)


Top 20 recommendations for user 12882:

1. Thai-style Pork Stew
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/45/45/2/3v8T842zQzSx5JtrrJ5A_download_20180403_202006.jpg

2. Nestle' Chewy Butterscotch Brownies
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/38/7/01481165638.jpeg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/38/7/picWlDqwX.jpg

3. Peanut Butter Cookies
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/10/26/9/vjcU2ElJQ1OownokVBCt_c6.jpg

4. Peanut Butter Balls
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/21/81/2/picAj4x43.jpg

5. Roast Beef and Gravy
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/48/73/9/picSzyvfH.jpg

In [ ]:
# Load the saved weights
# model.save_weights('/content/drive/MyDrive/Dataset/food_recommendation_model.h5')